# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once doc_to_synsets and similarity_score have been completed, submit to the autograder which will run a test to check that these functions are running correctly.

*Do not modify the functions `convert_tag` and `document_path_similarity`.*

In [1]:
%%capture
import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import wordnet as wn
import pandas as pd
nltk.data.path.append("assets/")

def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

In [33]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """

    tokens = nltk.word_tokenize(doc)
    word_tags = nltk.pos_tag(tokens)
    synsets = []
    for word, tag in word_tags:
        tag = convert_tag(tag)
        synset = wn.synsets(word, pos=tag)
        if len(synset) != 0:
            synsets.append(synset[0])
        else:
            continue
    return synsets


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.7333333333333333
    """

    largest_similarity_values = []
    for syn1 in s1:
        similarity_values = []
        for syn2 in s2: 
            sim_val = wn.path_similarity(syn1, syn2)
            if sim_val != None:
                similarity_values.append(sim_val)
            else:
                continue
        if len(similarity_values) != 0:
            largest_similarity_values.append(max(similarity_values))
    
    return sum(largest_similarity_values)/len(largest_similarity_values)
    # return (largest_similarity_values, sum(largest_similarity_values)/len(largest_similarity_values))

doc_to_synsets("Cats are beautiful!")
doc_to_synsets("Dogs are ugly!")
similarity_score(doc_to_synsets("Cats are beautiful!"), doc_to_synsets("Dogs are beautiful!"))

0.7333333333333334

In [34]:
doc_to_synsets("Cats are beautiful!")

[Synset('cat.n.01'), Synset('be.v.01'), Synset('beautiful.a.01')]

In [3]:
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [4]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('assets/paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [5]:
def most_similar_docs():
    
    docs1 = paraphrases['D1']
    docs2 = paraphrases['D2']
    
    docs = list(zip(docs1, docs2))
    
    doc_similarity = []
    for doc in docs:
        doc_similarity.append(document_path_similarity(doc[0], doc[1]))
        
    similarity_score = max(doc_similarity)
    D1 = paraphrases.iloc[doc_similarity.index(max(doc_similarity))][1]
    D2 = paraphrases.iloc[doc_similarity.index(max(doc_similarity))][2]
    
    return (D1, D2, similarity_score)

most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9590643274853801)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [6]:
def similarity():
    
    docs1 = paraphrases['D1']
    docs2 = paraphrases['D2']
    
    docs = list(zip(docs1, docs2))
    
    doc_similarity = []
    for doc in docs:
        doc_similarity.append(document_path_similarity(doc[0], doc[1]))
    
    return doc_similarity

similarity()

[0.645735766758494,
 0.8706068033273915,
 0.8320075757575757,
 0.6829947829947829,
 0.9301413255360624,
 0.8286604225023343,
 0.7333799302549302,
 0.6707575757575758,
 0.6083281797567511,
 0.7284470646312752,
 0.530408744694459,
 0.6277167277167277,
 0.5979593918987858,
 0.9590643274853801,
 0.6921006944444443,
 0.843720238095238,
 0.47432896964146964,
 0.536288616686344,
 0.5807763143289459,
 0.7193047337278107]

In [7]:
def label_accuracy():
    from sklearn.metrics import accuracy_score
    import numpy as np

    paraphrases2 = paraphrases
    paraphrases2["similarity_score"] = similarity()
    paraphrases2["Quality_new"] = np.where(paraphrases2['similarity_score'] > 0.75, 1, 0)
    
    return accuracy_score(paraphrases2['Quality'], paraphrases2['Quality_new'])

label_accuracy()

0.7

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [8]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('assets/newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [9]:
# HOW DOES A CORPUS LOOK LIKE?

# Document 1: (word_id, word_count) pairs
# Document 2: (word_id, word_count) pairs

In [21]:
sorted(id_map.items(), reverse=True)

[(900, 'young'),
 (899, 'york'),
 (898, 'yes'),
 (897, 'years'),
 (896, 'year'),
 (895, 'yeah'),
 (894, 'wrong'),
 (893, 'written'),
 (892, 'write'),
 (891, 'wouldn'),
 (890, 'worth'),
 (889, 'worst'),
 (888, 'worse'),
 (887, 'world'),
 (886, 'works'),
 (885, 'working'),
 (884, 'worked'),
 (883, 'work'),
 (882, 'words'),
 (881, 'word'),
 (880, 'wondering'),
 (879, 'wonder'),
 (878, 'won'),
 (877, 'wish'),
 (876, 'winning'),
 (875, 'wings'),
 (874, 'windows'),
 (873, 'win'),
 (872, 'willing'),
 (871, 'wide'),
 (870, 'white'),
 (869, 'west'),
 (868, 'went'),
 (867, 'weight'),
 (866, 'weeks'),
 (865, 'week'),
 (864, 'ways'),
 (863, 'way'),
 (862, 'watch'),
 (861, 'wasn'),
 (860, 'washington'),
 (859, 'wants'),
 (858, 'wanted'),
 (857, 'want'),
 (856, 'wait'),
 (855, 'view'),
 (854, 'video'),
 (853, 'version'),
 (852, 'vehicle'),
 (851, 'various'),
 (850, 'value'),
 (849, 'usually'),
 (848, 'usual'),
 (847, 'using'),
 (846, 'uses'),
 (845, 'useful'),
 (844, 'used'),
 (843, 'use'),
 (842, '

In [11]:
sorted(vect.vocabulary_.items(), reverse=True)

[('young', 900),
 ('york', 899),
 ('yes', 898),
 ('years', 897),
 ('year', 896),
 ('yeah', 895),
 ('wrong', 894),
 ('written', 893),
 ('write', 892),
 ('wouldn', 891),
 ('worth', 890),
 ('worst', 889),
 ('worse', 888),
 ('world', 887),
 ('works', 886),
 ('working', 885),
 ('worked', 884),
 ('work', 883),
 ('words', 882),
 ('word', 881),
 ('wondering', 880),
 ('wonder', 879),
 ('won', 878),
 ('wish', 877),
 ('winning', 876),
 ('wings', 875),
 ('windows', 874),
 ('win', 873),
 ('willing', 872),
 ('wide', 871),
 ('white', 870),
 ('west', 869),
 ('went', 868),
 ('weight', 867),
 ('weeks', 866),
 ('week', 865),
 ('ways', 864),
 ('way', 863),
 ('watch', 862),
 ('wasn', 861),
 ('washington', 860),
 ('wants', 859),
 ('wanted', 858),
 ('want', 857),
 ('wait', 856),
 ('view', 855),
 ('video', 854),
 ('version', 853),
 ('vehicle', 852),
 ('various', 851),
 ('value', 850),
 ('usually', 849),
 ('usual', 848),
 ('using', 847),
 ('uses', 846),
 ('useful', 845),
 ('used', 844),
 ('use', 843),
 ('usa',

In [12]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=id_map, passes=25, random_state=34)


### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.013*"information"')`

for example.

*This function should return a list of tuples.*

In [13]:
ldamodel

In [14]:
def lda_topics():
    
    return list(ldamodel.show_topics(num_topics=10, num_words=10))

lda_topics()
    

[(0,
  '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'),
 (1,
  '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'),
 (2,
  '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'),
 (3,
  '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'),
 (4,
  '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'),
 (5,
  '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'),
 (6,
  '0.0

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [15]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [16]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

def topic_distribution():
    
    vect_transformed = vect.transform(new_doc)
    
    corpus_new = gensim.matutils.Sparse2Corpus(vect_transformed, documents_columns=False)
    
    return list(ldamodel[corpus_new])[0]

topic_distribution()

[(0, 0.020003108),
 (1, 0.020003324),
 (2, 0.020001281),
 (3, 0.49674824),
 (4, 0.020004038),
 (5, 0.020004129),
 (6, 0.020002972),
 (7, 0.020002645),
 (8, 0.020003129),
 (9, 0.34322715)]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [17]:
def topic_names():
    
    return ['Education', 'Science', 'Computers & IT', 'Religion', 'Automobiles', 'Sports', 'Health', 'Religion', 'Computers & IT', 'Science']